In [136]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import seaborn as sns
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

In [202]:
df = pd.read_csv('C:/Users/taihs/OneDrive/Documents/nlp/train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [203]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [204]:
pd.crosstab(df['keyword'],df['target'])

target,0,1
keyword,,
ablaze,23,13
accident,11,24
aftershock,34,0
airplane%20accident,5,30
ambulance,18,20
...,...,...
wounded,11,26
wounds,23,10
wreck,30,7


In [205]:
df.loc[(df['keyword'].notnull()) & (df['location'].notnull()), 'result'] = df['text'] + ' ' + df['keyword'] + ' ' + df['location']
df.loc[(df['keyword'].notnull()) & (df['location'].isnull()), 'result'] = df['text'] + ' ' + df['keyword']
df.loc[(df['keyword'].isnull()) & (df['location'].notnull()), 'result'] = df['text'] + ' ' + df['location']
df.loc[(df['keyword'].isnull()) & (df['location'].isnull()), 'result'] = df['text']

In [206]:
cv = CountVectorizer(max_features = 3000) 
X = cv.fit_transform(df['result']).toarray()

In [207]:
print(X)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [208]:
y = df['target']

In [209]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=8)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (6090, 3000) (6090,)
Test set: (1523, 3000) (1523,)


In [210]:
rf = RandomForestClassifier(n_estimators=300)
rf.fit(X_train,y_train)

RandomForestClassifier(n_estimators=300)

In [211]:
rf_predict = rf.predict(X_test)

print('f1 score', f1_score(y_test,rf_predict))
print('accuracy', accuracy_score(y_test,rf_predict))

f1 score 0.7119518486672399
accuracy 0.7800393959290873


In [212]:
xgb = XGBClassifier(n_estimators=300)
xgb.fit(X_train,y_train)

C:\Users\taihs\anaconda3\envs\new_env\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:10:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=300, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [213]:
xgb_predict = xgb.predict(X_test)

print('f1 score', f1_score(y_test,xgb_predict))
print('accuracy', accuracy_score(y_test,xgb_predict))

f1 score 0.7287581699346405
accuracy 0.7820091923834537


In [143]:
rnn_hidden_dim = 5
word_embedding_dim = 50
max_features = 3000
batch_size = 32
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=X_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [144]:
model_rnn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 50)          150000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 5)                 280       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6         
Total params: 150,286
Trainable params: 150,286
Non-trainable params: 0
_________________________________________________________________


In [145]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [146]:
model_rnn.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(X_test, y_test))

Epoch 1/10
191/191 [==============================] - 146s 764ms/step - loss: 0.6979 - accuracy: 0.5688 - val_loss: 0.6924 - val_accuracy: 0.5647
Epoch 2/10
191/191 [==============================] - 150s 785ms/step - loss: 0.6920 - accuracy: 0.5718 - val_loss: 0.6917 - val_accuracy: 0.5647
Epoch 3/10
191/191 [==============================] - 153s 800ms/step - loss: 0.6912 - accuracy: 0.5718 - val_loss: 0.6911 - val_accuracy: 0.5647
Epoch 4/10
191/191 [==============================] - 157s 823ms/step - loss: 0.6906 - accuracy: 0.5718 - val_loss: 0.6906 - val_accuracy: 0.5647
Epoch 5/10
191/191 [==============================] - 166s 871ms/step - loss: 0.6899 - accuracy: 0.5718 - val_loss: 0.6900 - val_accuracy: 0.5647
Epoch 6/10
191/191 [==============================] - 166s 867ms/step - loss: 0.6893 - accuracy: 0.5718 - val_loss: 0.6895 - val_accuracy: 0.5647
Epoch 7/10
191/191 [==============================] - 167s 874ms/step - loss: 0.6888 - accuracy: 0.5718 - val_loss: 0.6890 -

In [147]:
score, acc = model_rnn.evaluate(X_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

48/48 [==============================] - 5s 105ms/step - loss: 0.6879 - accuracy: 0.5647
Test score: 0.6878678202629089
Test accuracy: 0.564674973487854


In [214]:
df = pd.read_csv('C:/Users/taihs/OneDrive/Documents/nlp/test.csv')
df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [215]:
df.loc[(df['keyword'].notnull()) & (df['location'].notnull()), 'result'] = df['text'] + ' ' + df['keyword'] + ' ' + df['location']
df.loc[(df['keyword'].notnull()) & (df['location'].isnull()), 'result'] = df['text'] + ' ' + df['keyword']
df.loc[(df['keyword'].isnull()) & (df['location'].notnull()), 'result'] = df['text'] + ' ' + df['location']
df.loc[(df['keyword'].isnull()) & (df['location'].isnull()), 'result'] = df['text']

In [216]:
cv = CountVectorizer(max_features = 3000) 
X = cv.fit_transform(df['result']).toarray()


In [223]:
res = rf.predict(X)
#res = model_rnn.predict(X)

In [224]:
res

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [225]:
final = pd.DataFrame ({'target':res})

In [226]:
final = pd.concat([df['id'],final],axis=1)

In [227]:
final.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [228]:
final.to_csv('result.csv',index=False)